# Setup command line client

In [1]:
%env CLIENT bitrepository-client-1.9-RC1

!wget -Nq "https://sbforge.org/download/attachments/25395346/${CLIENT}.zip"
!unzip -quo ${CLIENT}.zip

%alias bitmag ${CLIENT}/bin/bitmag.sh %l

'/home/abr/Projects/jupyter-release-tests'

# Describes the tests needed to validate the GetChecksums functionality.

Upload a file with a known checksum for so we have something to test on

In [7]:
%env TESTFILE1 README.md

env: TESTFILE1=README.md


In [8]:
%bitmag put-file -c integrationtest1 -f $TESTFILE1 -i $TESTFILE1

Failure: a2bf9c03: PUT_FILE for file README.md: FAILED: , Failed operation. Cause(s):
[a2bf9c03: PUT_FILE for file README.md: COMPONENT_FAILED: ContributorID kbpillar2 ResponseCode: FILE_TRANSFER_FAILURE, File transfer for README.md to http://int-bitmag-01.kb.dk:80/dav/README.md]


## Basic getChecksum

In [9]:
!md5sum $TESTFILE1

e3815360b4db52d61191d985b79a208c  README.md


### Request the checksum of a file in the collection.

In [10]:
%bitmag get-checksums -c integrationtest1 -i $TESTFILE1

Count: 	Checksum: 	Pillars: 	FileID: 
4 	e3815360b4db52d61191d985b79a208c 	All 	README.md


The correct checksum should be returned by all the pillars.

## getChecksum (salted)

### Request the salted checksum of a file in the collection

In [12]:
import hmac
import hashlib
import base64

def getSaltedChecksum(file,saltHex,algorithm):
    #Note, the salt is a hex string
    salt = bytearray.fromhex(saltHex)
    digester = hmac.new(salt,None,algorithm)
    with open(file,'rb') as from_fh:
        while True:
            chunk = from_fh.read()
            if not chunk:
                break
            digester.update(chunk)
    return digester.hexdigest().lower()

file = os.environ.get('TESTFILE1')
print(getSaltedChecksum(file,saltHex='abcd',algorithm=hashlib.md5))

7ecaa01429e95892e3353be8356c9b40


In [14]:
%bitmag get-checksums -c integrationtest1 -i $TESTFILE1 -R HMAC_MD5 -S 'abcd'

Count: 	Checksum: 	Pillars: 	FileID: 
3 	7ecaa01429e95892e3353be8356c9b40 	All 	README.md


The correct salted checksum should be returned by all the pillars (except the ChecksumPillar).

## Missing file

Request the checksum of a file that does not exist in the collection.

In [16]:
%bitmag get-checksums -c integrationtest1 -i ThisFileDoesNotExist

Count: 	Checksum: 	Pillars: 	FileID: 


No results should be returned.